In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd /home/aditya/git/kaggle_pneumonia 
%env PROJECT_PATH = /home/aditya/git/kaggle_pneumonia 

/home/aditya/git/kaggle_pneumonia
env: PROJECT_PATH=/home/aditya/git/kaggle_pneumonia


In [3]:
import pandas as pd
import numpy as np
import pydicom
from PIL import Image
import multiprocessing as mp
from tqdm import tqdm_notebook as tqdm

In [4]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [6]:
from envs import *
from utils.data_load import *
import torchvision

from model import PneuNet, PneuNetHeader
from dataset import Single_Pneumonia_Dataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
from loss import BoundBoxCriterion, train

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
train_label_df = pd.read_csv(train_label_repo)
train_label_df['area'] = train_label_df.width * train_label_df.height
single_label_df = train_label_df.sort_values('area', ascending = False).drop_duplicates('patientId').sort_index().reset_index(drop = True).copy()
single_label_df.fillna(0, inplace = True)

In [9]:
train_idx = np.arange(len(single_label_df))
dev_idx, val_idx = train_test_split(train_idx, test_size = 0.20)

In [10]:
dev_df = single_label_df.iloc[dev_idx,:].reset_index(drop = True)
val_df = single_label_df.iloc[val_idx,:].reset_index(drop = True)

In [11]:
dev_dataset = Single_Pneumonia_Dataset(dev_df, train_dcm_path, device)
val_dataset = Single_Pneumonia_Dataset(val_df, train_dcm_path, device)

In [12]:
dev_dataloader = DataLoader(dev_dataset, batch_size = 64, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 64)

In [16]:
preload_model = torchvision.models.densenet121(pretrained=True).to(device)
header_model = PneuNetHeader(1024, 5, [1000, 1000, 128], 0.2).to(device)
pneunet_model = PneuNet(preload_model, header_model).to(device)

In [17]:
criterion = BoundBoxCriterion(20.).to(device)
optimizer = optim.Adam(pneunet_model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [18]:
pneunet_model.train()
n_epoch = 50
for idx_epoch in tqdm(range(n_epoch), total = n_epoch):
    t = tqdm(enumerate(dev_dataloader), total = 322)
    for idx, data in t:
        img, target = data
        pred = pneunet_model(img)
        loss, loss_label, loss_bb = criterion(pred, target)
        train(optimizer, loss)
        t.set_description("loss : {}".format(loss))
    print(loss_label, loss_bb)

tensor(0.5154, device='cuda:0', grad_fn=<MeanBackward1>) tensor(nan, device='cuda:0', grad_fn=<SmoothL1LossBackward>)


tensor(0.3042, device='cuda:0', grad_fn=<MeanBackward1>) tensor(nan, device='cuda:0', grad_fn=<SmoothL1LossBackward>)


KeyboardInterrupt: 